# Hollender 3D image analysis
## Author: Miles Roberts
## Last updated: 2021-04-02
## Goals:
* Read in .ply files
* Process images to get baseline skeletons that all subgroups will work with
* Measure cool phenotypes
## Link to cookbook: http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#

## Bring methods together to isolate plant

### Try downsampling before plane segmentation

In [1]:
import numpy as np
import open3d as o3d

# Read .ply file
input_file = "./Hollender_arabidopsis/arabidopsis lab test 4 (col).ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Visualize the point cloud within open3d
#o3d.visualization.draw_geometries([pcd]) 

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
#point_cloud_in_numpy = np.asarray(pcd.points) 

#Downsampling
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
o3d.visualization.draw_geometries([downpcd])

#Identify plane in downsampled figure
plane_model, inliers = downpcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = downpcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = downpcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

Plane equation: 0.01x + -0.03y + 1.00z + 0.55 = 0


## Remove segmented plane

In [2]:
#Write function to take pcd object and plane formula then subset points
def remove_plane(pcdObject, modelArray, upr, lwr):
    #Write list for output
    goodIndices = []
    #Convert pcd object to numpy array, save points; normals; and colors
    pcdPoints = np.asarray(pcdObject.points)
    pcdNormals = np.asarray(pcdObject.normals)
    pcdColors = np.asarray(pcdObject.colors)
    #Extract coefficients for plane equation
    a = plane_model[0]
    b = plane_model[1]
    c = plane_model[2]
    d = plane_model[3]
    #evaluate plane equation for each point
    for i in range(len(pcdPoints)):
        x = pcdPoints[i][0]
        y = pcdPoints[i][1]
        z = pcdPoints[i][2]
        if(a*x + b*y + c*z + d < upr and a*x + b*y + c*z + d > lwr):
            goodIndices.append(i)
    #subset array, convert array back to cloud
    pcdNew = o3d.geometry.PointCloud()
    pcdNew.points = o3d.utility.Vector3dVector(pcdPoints[goodIndices])
    pcdNew.normals = o3d.utility.Vector3dVector(pcdNormals[goodIndices])
    pcdNew.colors = o3d.utility.Vector3dVector(pcdColors[goodIndices])
    return pcdNew

In [3]:
noPlanePcd = remove_plane(downpcd, plane_model, 0.1, 0.005)
o3d.visualization.draw_geometries([noPlanePcd])

## Crop along the other axes to isolate the plant

In [4]:
def remove_xzplane(pcdObject, lwr, upr):
    #Write list for output
    goodIndices = []
    #Convert pcd object to numpy array, save points; normals; and colors
    pcdPoints = np.asarray(pcdObject.points)
    pcdNormals = np.asarray(pcdObject.normals)
    pcdColors = np.asarray(pcdObject.colors)
    #evaluate plane equation for each point
    for i in range(len(pcdPoints)):
        x = abs(pcdPoints[i][0])
        if(x < upr and x > lwr):
            goodIndices.append(i)
    #subset array, convert array back to cloud
    pcdNew = o3d.geometry.PointCloud()
    pcdNew.points = o3d.utility.Vector3dVector(pcdPoints[goodIndices])
    pcdNew.normals = o3d.utility.Vector3dVector(pcdNormals[goodIndices])
    pcdNew.colors = o3d.utility.Vector3dVector(pcdColors[goodIndices])
    return pcdNew

In [5]:
noXZplanePcd = remove_xzplane(noPlanePcd, 0, 0.15)
o3d.visualization.draw_geometries([noXZplanePcd])

In [6]:
#Write function to take pcd object and plane formula then subset points
def remove_xyplane(pcdObject, lwr, upr):
    #Write list for output
    goodIndices = []
    #Convert pcd object to numpy array, save points; normals; and colors
    pcdPoints = np.asarray(pcdObject.points)
    pcdNormals = np.asarray(pcdObject.normals)
    pcdColors = np.asarray(pcdObject.colors)
    #evaluate plane equation for each point
    for i in range(len(pcdPoints)):
        y = pcdPoints[i][1]
        if(y < upr and y > lwr):
            goodIndices.append(i)
    #subset array, convert array back to cloud
    pcdNew = o3d.geometry.PointCloud()
    pcdNew.points = o3d.utility.Vector3dVector(pcdPoints[goodIndices])
    pcdNew.normals = o3d.utility.Vector3dVector(pcdNormals[goodIndices])
    pcdNew.colors = o3d.utility.Vector3dVector(pcdColors[goodIndices])
    return pcdNew

In [11]:
Coord = np.asarray(noXZplanePcd.points)
yCoord = [Coord[i][1] for i in range(len(Coord))]
yCoord

[-0.12228012084960938,
 -0.146209716796875,
 -0.1258544921875,
 -0.15643310546875,
 0.17085774739583334,
 -0.117828369140625,
 -0.030645751953125,
 -0.0238189697265625,
 0.06640625,
 0.18299696180555555,
 0.217974853515625,
 0.217816162109375,
 -0.160400390625,
 -0.04437255859375,
 -0.03708521525065104,
 0.218170166015625,
 0.2229546440972222,
 -0.11029052734375,
 -0.13222369025735295,
 -0.028472900390625,
 0.2075653076171875,
 -0.116943359375,
 -0.129638671875,
 -0.11383056640625,
 -0.16829427083333334,
 0.019846598307291668,
 -0.144287109375,
 -0.170849609375,
 -0.12784830729166666,
 0.223358154296875,
 0.22325942095588236,
 -0.11372884114583333,
 -0.10765584309895833,
 0.022823627178485576,
 0.207965087890625,
 -0.08371988932291667,
 -0.08240215594951923,
 0.21315765380859375,
 0.19923909505208334,
 0.22278594970703125,
 -0.11728922526041667,
 0.19334193638392858,
 0.07080078125,
 0.091644287109375,
 -0.16644287109375,
 0.22785186767578125,
 0.04730224609375,
 -0.11887613932291667,


In [12]:
min(yCoord)

-0.1766357421875

In [13]:
max(yCoord)

0.231597900390625

In [14]:
noXYplanePcd = remove_xyplane(noXZplanePcd, -0.03, 0.21)
o3d.visualization.draw_geometries([noXYplanePcd])

## Finally, Use DBSCAN clustering to find largest point cloud (i.e. the plant)

In [15]:
from matplotlib import pyplot as plt 
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(noXYplanePcd.cluster_dbscan(eps=0.08, min_points=300, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
noXYplanePcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([noXYplanePcd])

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 1
point cloud has 1 clusters


In [16]:
finalPoints = np.asarray(noXYplanePcd.points)[np.where(labels==0)]

finalPcd = o3d.geometry.PointCloud()
finalPcd.points = o3d.utility.Vector3dVector(finalPoints)

o3d.visualization.draw_geometries([finalPcd])

## Skeletonize with sci-kit image

In [45]:
np.asarray(finalPcd.points).shape

(704, 3)

In [44]:
np.zeros((2, 3, 4))

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]])

In [43]:
from skimage import morphology

skel = morphology.skeletonize_3d(np.asarray(finalPcd.points))

array([[ 0, 51,  0],
       [ 0,  0,  0],
       [ 0, 51,  0],
       ...,
       [ 0,  0,  0],
       [51,  0,  0],
       [51,  0,  0]], dtype=uint8)

# GRAVEYARD

In [ ]:
#Write function to compare statistical outlier removal to original point cloud
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
noXYplanePcd, ind = noXYplanePcd.remove_statistical_outlier(nb_neighbors=20,std_ratio=1)
display_inlier_outlier(noXYplanePcd, ind)

In [ ]:
def segment_color(pcdObject, colorChoice):
    #Write list for output
    goodIndices = []
    #Convert pcd object to numpy array, save points; normals; and colors
    pcdPoints = np.asarray(pcdObject.points)
    pcdNormals = np.asarray(pcdObject.normals)
    pcdColors = np.asarray(pcdObject.colors)
    #Extract color bounds
    rlwr = colorChoice[0]
    rupr = colorChoice[1]
    glwr = colorChoice[2]
    gupr = colorChoice[3]
    blwr = colorChoice[4]
    bupr = colorChoice[5]
    #evaluate plane equation for each point
    for i in range(len(pcdColors)):
        r = pcdColors[i][0]
        g = pcdColors[i][1]
        b = pcdColors[i][2]
        if(r > rlwr and r < rupr and g > glwr and g < gupr and b > blwr and b < bupr):
            goodIndices.append(i)
    #subset array, convert array back to cloud
    pcdNew = o3d.geometry.PointCloud()
    pcdNew.points = o3d.utility.Vector3dVector(pcdPoints[goodIndices])
    pcdNew.normals = o3d.utility.Vector3dVector(pcdNormals[goodIndices])
    pcdNew.colors = o3d.utility.Vector3dVector(pcdColors[goodIndices])
    return pcdNew

In [ ]:
noColorPcd = segment_color(noOutPcd, [0,1,0,1,0,1])
o3d.visualization.draw_geometries([noColorPcd])

### Try not downsampling before plane segmentation

In [ ]:
#Identify plane in downsampled figure
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
noPlanePcd = remove_plane(pcd, plane_model, 0.1, 0.005)
o3d.visualization.draw_geometries([noPlanePcd])